<a href="https://colab.research.google.com/github/mandliya/ml/blob/master/hands_on_machine_learning/2/End_to_end_machine_learning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End to End Machine Learning Project

In this project we will go through an example project end to end, pretending to be a recently hired data scientist in a real estate company.Here are the main steps you will go through:
<ul>
<li>Look at the big picture.</li>
<li>Get the data.</li>
<li>Discover and visualize the data to gain insights.</li>
<li>Prepare the data for Machine Learning algorithms.</li>
<li>Select a model and train it.</li>
<li>Fine-tune your model.</li>
<li>Present your solution.</li>
<li>Launch, monitor, and maintain your system.</li>
</ul>